# Installing Packages

In [1]:
%%capture
!pip install rouge_score evaluate bert_score underthesea

# Import Libs

In [2]:
import numpy as np
import pandas as pd

import evaluate
from bert_score import score
from underthesea import word_tokenize
from tqdm.auto import tqdm

2025-06-26 04:10:24.551232: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750911024.836507      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750911024.910879      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Scoring

In [3]:
rouge = evaluate.load("rouge")
bleu = evaluate.load("bleu")

In [4]:
def tokenize(text:str):
    return word_tokenize(text, format="text")

def get_rouge(prediction:str, reference:str):
    return rouge.compute(predictions=[tokenize(prediction)], references=[tokenize(reference)])

def get_bleu(prediction:str, reference:str):
    return bleu.compute(predictions=[tokenize(prediction)], references=[[tokenize(reference)]])

def get_bert_score(prediction:str, reference:str):
    P, R, F1 = score([tokenize(prediction)], [tokenize(reference)], lang="vi")
    return F1.mean().item()

In [5]:
# For Example
get_bert_score(
    "Ở sân chơi đẳng cấp nhất thế giới như World Cup, những ngôi sao kỳ cựu của đội tuyển nữ Việt Nam",
    "Tại World Cup 2023, đội tuyển nữ Việt Nam gặp rất nhiều khó khăn vì Covid-19 và sức khỏe của các cầu thủ"
)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

0.7592025399208069

# Load Data

In [6]:
pretrained_df = pd.read_csv("/kaggle/input/news-summary-testing-pretrain-qwen/pretrain_result.csv")
print(pretrained_df.isnull().sum())
print(pretrained_df.duplicated().sum())
pretrained_df.head(3)

Content             0
Summary             0
Pretrain_summary    0
dtype: int64
0


,Content,Summary,Pretrain_summary
0,Huỳnh Như (9) và Tuyết Dung (7) là cả kho tàng...,Ở sân chơi đẳng cấp nhất thế giới như World Cu...,Huỳnh Như và Tuyết Dung là cả kho tàng kinh ng...
1,Chiến công mới của các anh công an thật tuyệt ...,Việc công an nhanh chóng bắt được 2 nghi phạm ...,"Trong ngày 27/5, các lực lượng công an đã thực..."
2,"""Tôi rất vui và phấn khởi, cũng có đôi chút hồ...","Ngay ở lần tham dự Á vận hội đầu tiên, đội tu...","Trần Nguyên Hưng, VĐV đội tuyển Liên Minh Huyề..."


In [7]:
finetuned_df = pd.read_csv("/kaggle/input/news-summary-testing-finetuned-qwen/finetuned_result.csv")
print(finetuned_df.isnull().sum())
print(finetuned_df.duplicated().sum())
finetuned_df.head(3)

Content              0
Summary              0
Finetuned_summary    0
dtype: int64
0


,Content,Summary,Finetuned_summary
0,Huỳnh Như (9) và Tuyết Dung (7) là cả kho tàng...,Ở sân chơi đẳng cấp nhất thế giới như World Cu...,"Tại World Cup 2023, đội tuyển nữ Việt Nam gặp ..."
1,Chiến công mới của các anh công an thật tuyệt ...,Việc công an nhanh chóng bắt được 2 nghi phạm ...,"Trước tình trạng cướp, giết người, nạn nhân ph..."
2,"""Tôi rất vui và phấn khởi, cũng có đôi chút hồ...","Ngay ở lần tham dự Á vận hội đầu tiên, đội tu...",Trong trận chung kết vòng loại World Cup 2024 ...


# Processing

In [8]:
def process_df(df, predict_col="Pretrain_summary"):
    rouge1 = []
    rouge2 = []
    rougeL = []
    rougeLsum = []
    bertScore = []

    for i in tqdm(range(0, df.shape[0]), desc="Processing"):
        reference = df["Summary"].iloc[i]
        prediction = df[predict_col].iloc[i]
        r = get_rouge(prediction, reference)
        rouge1.append(r["rouge1"])
        rouge2.append(r["rouge2"])
        rougeL.append(r["rougeL"])
        rougeLsum.append(r["rougeLsum"])
        bertScore.append(get_bert_score(prediction, reference))

    df.loc[:, [
        "rouge1", "rouge2", "rougeL", "rougeLsum",
        "bertScore"
    ]] = pd.DataFrame({
        "rouge1": rouge1,
        "rouge2": rouge2,
        "rougeL": rougeL,
        "rougeLsum": rougeLsum,
        "bertScore": bertScore
    })
    return df

pretrained_df = process_df(pretrained_df, predict_col="Pretrain_summary")
finetuned_df = process_df(finetuned_df, predict_col="Finetuned_summary")

Processing:   0%|          | 0/2000 [00:00<?, ?it/s]

Processing:   0%|          | 0/2000 [00:00<?, ?it/s]

In [9]:
pretrained_df.head()

,Content,Summary,Pretrain_summary,rouge1,rouge2,rougeL,rougeLsum,bertScore
0,Huỳnh Như (9) và Tuyết Dung (7) là cả kho tàng...,Ở sân chơi đẳng cấp nhất thế giới như World Cu...,Huỳnh Như và Tuyết Dung là cả kho tàng kinh ng...,0.454902,0.268775,0.266667,0.266667,0.750646
1,Chiến công mới của các anh công an thật tuyệt ...,Việc công an nhanh chóng bắt được 2 nghi phạm ...,"Trong ngày 27/5, các lực lượng công an đã thực...",0.385965,0.158824,0.245614,0.245614,0.686364
2,"""Tôi rất vui và phấn khởi, cũng có đôi chút hồ...","Ngay ở lần tham dự Á vận hội đầu tiên, đội tu...","Trần Nguyên Hưng, VĐV đội tuyển Liên Minh Huyề...",0.506494,0.092105,0.285714,0.285714,0.690039
3,"Sau hai tháng liên tiếp sụt giảm, mới đây sức ...",Sức mua ô tô trên thị trường vừa có dấu hiệu h...,"Đầu tháng 10/2023, thị trường ô tô Việt Nam đã...",0.530035,0.277580,0.289753,0.289753,0.727363
4,"Theo đó, mức lãi suất cho vay ưu đãi tại Ngân ...",Thủ tướng đã ký Quyết định 486 về mức lãi suất...,Dưới đây là cách tóm tắt văn bản của bạn:\n\n-...,0.537931,0.250000,0.296552,0.296552,0.703700


In [10]:
finetuned_df.head()

,Content,Summary,Finetuned_summary,rouge1,rouge2,rougeL,rougeLsum,bertScore
0,Huỳnh Như (9) và Tuyết Dung (7) là cả kho tàng...,Ở sân chơi đẳng cấp nhất thế giới như World Cu...,"Tại World Cup 2023, đội tuyển nữ Việt Nam gặp ...",0.600000,0.202899,0.285714,0.285714,0.720319
1,Chiến công mới của các anh công an thật tuyệt ...,Việc công an nhanh chóng bắt được 2 nghi phạm ...,"Trước tình trạng cướp, giết người, nạn nhân ph...",0.473684,0.071429,0.245614,0.245614,0.692962
2,"""Tôi rất vui và phấn khởi, cũng có đôi chút hồ...","Ngay ở lần tham dự Á vận hội đầu tiên, đội tu...",Trong trận chung kết vòng loại World Cup 2024 ...,0.356436,0.121212,0.237624,0.237624,0.701569
3,"Sau hai tháng liên tiếp sụt giảm, mới đây sức ...",Sức mua ô tô trên thị trường vừa có dấu hiệu h...,Số lượng xe bán trên thị trường Việt Nam trong...,0.390977,0.122137,0.225564,0.225564,0.664570
4,"Theo đó, mức lãi suất cho vay ưu đãi tại Ngân ...",Thủ tướng đã ký Quyết định 486 về mức lãi suất...,Một số ngân hàng thương mại đã công bố gói tín...,0.687898,0.348387,0.407643,0.407643,0.771276


# Save data

In [11]:
pretrained_df.to_csv("pretrained_score.csv", index=False)
finetuned_df.to_csv("finetuned_score.csv", index=False)